<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/Model_Selector_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# MODEL SELECTOR – COLAB E2E
# =========================

# === IMPORTS (Colab + Data) ===
import os
import sys
import ast
import json
import argparse
import logging
from datetime import datetime
from pathlib import Path
from typing import Optional, Tuple, Dict, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# If running on Colab, import and mount Drive
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    # === MOUNT GOOGLE DRIVE ===
    drive.mount('/content/drive', force_remount=True)

# === CONFIGURATION (adjust paths to your Drive folder) ===
SELECTOR_DIR = "/content/drive/MyDrive/Model_Selector/Result" if IN_COLAB else "./Result"
COMBINED_CSV = os.path.join(SELECTOR_DIR, "all_models_selector_ready.csv")
OUTDIR = os.path.join(SELECTOR_DIR, "out")  # will be created if missing

# =========================
# STEP A: PREP INPUT (gather *_model_selector.csv and combine)
# =========================
os.makedirs(SELECTOR_DIR, exist_ok=True)
model_files = [f for f in os.listdir(SELECTOR_DIR) if f.endswith('_model_selector.csv')]
model_dfs = []

for file in model_files:
    file_path = os.path.join(SELECTOR_DIR, file)
    try:
        df = pd.read_csv(file_path)
        # If 'Model' column not set, derive from filename:
        if "Model" not in df.columns:
            df["Model"] = file.replace("_model_selector.csv", "")
        model_dfs.append(df)
        print(f" Loaded: {file}")
    except Exception as e:
        print(f" Error loading {file}: {e}")

if not model_dfs:
    raise ValueError("No model selector CSVs found in folder. "
                     "Place files ending with *_model_selector.csv into SELECTOR_DIR.")

# Combine and save the master CSV (used by the selector pipeline below)
master_df = pd.concat(model_dfs, ignore_index=True)
master_df.to_csv(COMBINED_CSV, index=False)
print(f"\n📂 Combined model selector results saved to:\n{COMBINED_CSV}")

# Optional quick cleaning in this prep stage (safe and minimal)
df_pre = master_df.copy()
# Fix 'Signal Map' if exists (stringified dicts)
if 'Signal Map' in df_pre.columns:
    df_pre['Signal Map'] = df_pre['Signal Map'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("{") else x
    )
# Drop legacy 'score' (lowercase) if present to avoid confusion with 'Score'
if 'score' in df_pre.columns:
    df_pre.drop(columns=['score'], inplace=True)
print(f"📐 Combined shape: {df_pre.shape}")

# =========================
# STEP B: SELECTOR PIPELINE (full, with argparse + try/except)
# =========================

# -------------------------------- logging --------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
log = logging.getLogger("selector")
try:
    from scipy.stats import wilcoxon  # noqa: F401
except Exception:
    log.warning("SciPy not available; falling back to mean-edge rule for incumbent significance.")

import operator as _op

# -------------------------------- utils ----------------------------------

def parse_weights(s: str) -> Dict[str, float]:
    out: Dict[str, float] = {}
    if not s:
        return out
    for part in s.split(","):
        if "=" not in part:
            continue
        k, v = part.split("=", 1)
        try:
            out[k.strip()] = float(v.strip())
        except ValueError:
            pass
    return out

def winsorize(series: pd.Series, p=0.01) -> pd.Series:
    if p <= 0:
        return series
    lo, hi = series.quantile(p), series.quantile(1 - p)
    return series.clip(lower=lo, upper=hi)

def zscore(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce").astype(float)
    mu = s.mean()
    sd = s.std(ddof=0)
    if not np.isfinite(sd) or sd == 0:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd

def robust_z(series: pd.Series) -> pd.Series:
    x = pd.to_numeric(series, errors="coerce")
    med = x.median()
    mad = (x - med).abs().median()
    if not np.isfinite(mad) or mad == 0:
        return pd.Series(np.zeros(len(x)), index=x.index)
    return 0.6745 * (x - med) / mad

def coerce_numeric(df: pd.DataFrame, cols) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def compute_score_frame(df: pd.DataFrame, weights: Dict[str, float], scale_mode="z", winsor_p=0.0) -> pd.DataFrame:
    metrics = [m for m in ["Sharpe", "Return", "Accuracy", "Drawdown", "HitRatio", "WinRate", "Final_Portfolio", "Turnover"]
               if m in df.columns]
    df = coerce_numeric(df, metrics)
    if winsor_p > 0:
        for m in metrics:
            df[m] = winsorize(df[m], winsor_p)
    scaler = robust_z if scale_mode.lower() == "robust" else (zscore if scale_mode.lower() == "z" else None)
    scaled: Dict[str, pd.Series] = {}
    for m in metrics:
        v = df[m].fillna(0.0)
        scaled[m] = scaler(v) if scaler else v
    score = pd.Series(0.0, index=df.index)
    for m, s in scaled.items():
        w = float(weights.get(m, 0.0))
        score = score + w * s
    df["Score"] = pd.to_numeric(score, errors="coerce").fillna(0.0)
    return df

# ---------- universal helpers (family, annualization, resources, family-weights) ----------

def infer_family_from_model(name: str) -> str:
    n = (name or "").strip().lower()
    rl = {"ppo","a2c","ddpg","td3","sac","dqn","deep sarsa","sarsa","reinforce"}
    tree = {"xgboost","xgb","lightgbm","lgbm","randomforest","random_forest","rf","catboost","cb"}
    cluster = {"kmeans","k-means","k_means"}
    if any(k in n for k in rl): return "RL"
    if any(k in n for k in tree): return "Tree"
    if any(k in n for k in cluster): return "Cluster"
    return "Other"

def harmonize_accuracy_like(df: pd.DataFrame) -> pd.DataFrame:
    if "Accuracy" not in df.columns:
        if "HitRatio" in df.columns:
            df["Accuracy"] = pd.to_numeric(df["HitRatio"], errors="coerce")
        elif "WinRate" in df.columns:
            df["Accuracy"] = pd.to_numeric(df["WinRate"], errors="coerce")
    return df

def annualize_metrics(df: pd.DataFrame, period_col: str, trading_days: int) -> pd.DataFrame:
    if period_col in df.columns:
        pdays = pd.to_numeric(df[period_col], errors="coerce").replace(0, np.nan)
        if "Sharpe" in df.columns:
            df["Sharpe"] = pd.to_numeric(df["Sharpe"], errors="coerce") * np.sqrt(trading_days / pdays)
        if "Return" in df.columns:
            r = pd.to_numeric(df["Return"], errors="coerce")
            df["Return"] = np.sign(r) * (np.power(1.0 + r.abs(), trading_days / pdays) - 1.0)
    return df

def apply_resource_gates(df: pd.DataFrame, require_artifacts: bool, max_latency_ms: int, max_mem_mb: int) -> pd.DataFrame:
    m = pd.Series(True, index=df.index)
    if require_artifacts:
        have_any = False
        for c in ["artifact_path","vecnorm_path","features_path"]:
            if c in df.columns:
                have_any = True
                m &= df[c].notna() & (df[c].astype(str).str.len() > 0)
        if not have_any:
            pass  # don't drop everything if columns don't exist
    if max_latency_ms > 0 and "latency_ms" in df.columns:
        m &= pd.to_numeric(df["latency_ms"], errors="coerce") <= max_latency_ms
    if max_mem_mb > 0 and "mem_mb" in df.columns:
        m &= pd.to_numeric(df["mem_mb"], errors="coerce") <= max_mem_mb
    out = df[m].fillna(False)
    out = df[out]
    return out if not out.empty else df

def parse_family_weights(s: str) -> dict:
    if not s: return {}
    try:
        j = json.loads(s)
        out = {}
        for fam, spec in j.items():
            if isinstance(spec, str):
                out[fam] = parse_weights(spec)
            elif isinstance(spec, dict):
                out[fam] = {k: float(v) for k, v in spec.items()}
        return out
    except Exception as e:
        log.warning("Failed to parse --family_weights JSON: %s", e)
        return {}

def compute_score_frame_with_family_weights(df: pd.DataFrame,
                                            default_weights: dict,
                                            family_weights: dict,
                                            scale_mode="z",
                                            winsor_p=0.0) -> pd.DataFrame:
    metrics = [m for m in ["Sharpe","Return","Accuracy","Drawdown","HitRatio","WinRate","Final_Portfolio","Turnover"]
               if m in df.columns]
    df = coerce_numeric(df, metrics)
    if winsor_p > 0:
        for m in metrics:
            df[m] = winsorize(df[m], winsor_p)
    scaler = robust_z if scale_mode.lower() == "robust" else (zscore if scale_mode.lower() == "z" else None)
    scaled = {}
    for m in metrics:
        v = df[m].fillna(0.0)
        scaled[m] = scaler(v) if scaler else v
    S = pd.DataFrame({m: scaled[m] for m in metrics}, index=df.index)
    fam = df["Family"] if "Family" in df.columns else df["Model"].astype(str).apply(infer_family_from_model)
    scores = pd.Series(0.0, index=df.index)
    for i in df.index:
        f = fam.loc[i]
        w = family_weights.get(f, default_weights)
        val = 0.0
        for m in metrics:
            if m in S.columns:
                val += float(w.get(m, 0.0)) * float(S.loc[i, m])
        scores.loc[i] = val
    df["Score"] = pd.to_numeric(scores, errors="coerce").fillna(0.0)
    return df

# -------------------------------- gates / schema / hygiene --------------------------------
_OPS = {"<=":_op.le, ">=":_op.ge, "<":_op.lt, ">":_op.gt, "==":_op.eq, "!=":_op.ne}

def apply_gates_expr(df: pd.DataFrame, spec: str) -> pd.DataFrame:
    if not spec: return df
    mask = pd.Series(True, index=df.index)
    for tok in spec.split(","):
        tok = tok.strip()
        if not tok: continue
        for sym in ["<=",">=","!=","==","<",">"]:
            if sym in tok:
                col, val = tok.split(sym, 1)
                col, val = col.strip(), val.strip()
                if col in df.columns:
                    v_num = pd.to_numeric(df[col], errors="coerce")
                    try:
                        fval = float(val)
                        mask &= _OPS[sym](v_num, fval)
                    except ValueError:
                        if sym in ("==", "!="):
                            val_clean = val.strip().strip('"\'')
                            v_str = df[col].astype(str)
                            left  = v_str.str.lower()
                            right = val_clean.lower()
                            mask &= (left == right) if sym == "==" else (left != right)
                break
    out = df[mask.fillna(False)]
    return out if not out.empty else df

def validate_schema(df: pd.DataFrame):
    REQUIRED_ANY = [["Ticker","Symbol"], ["Model"]]
    OPTIONAL = {"Sharpe","Return","Accuracy","Drawdown","Max_Drawdown","Sortino",
                "HitRatio","WinRate","Final_Portfolio","Turnover","Trade_Count",
                "EndDate","artifact_path","vecnorm_path","features_path",
                "latency_ms","mem_mb","regime","rl_profile","PeriodDays"}
    for group in REQUIRED_ANY:
        if not any(col in df.columns for col in group):
            raise SystemExit(f"Missing one of required columns: {group}")
    unknown = [c for c in df.columns if c not in OPTIONAL and c not in {"Ticker","Symbol","Model","Family"}]
    if unknown:
        log.info("Extra columns (ok): %s", ", ".join(sorted(unknown)))

def normalize_drawdown_sign(df: pd.DataFrame) -> pd.DataFrame:
    for c in ["Drawdown","Max_Drawdown","MaxDD_Mean"]:
        if c in df.columns:
            x = pd.to_numeric(df[c], errors="coerce")
            if (x.dropna() > 0).mean() > 0.8:
                df[c] = -x
    return df

def dedupe(df: pd.DataFrame) -> pd.DataFrame:
    if {"Ticker","Model"}.issubset(df.columns):
        before = len(df)
        df = (df.sort_values("Score", ascending=False)
                .drop_duplicates(["Ticker","Model"], keep="first"))
        if len(df) < before:
            log.info("Deduped (Ticker,Model): %d -> %d", before, len(df))
    return df

# -------------------------------- safety gates --------------------------------

def apply_safety_gates(df: pd.DataFrame) -> pd.DataFrame:
    maxdd_col = "Max_Drawdown" if "Max_Drawdown" in df.columns else ("Drawdown" if "Drawdown" in df.columns else None)
    m = pd.Series(True, index=df.index)
    if "Sharpe" in df.columns:
        m &= pd.to_numeric(df["Sharpe"], errors="coerce") > 0.0
    if maxdd_col:
        m &= pd.to_numeric(df[maxdd_col], errors="coerce") > -0.50
    if "Return" in df.columns:
        m &= pd.to_numeric(df["Return"], errors="coerce") > -0.20
    out = df[m].fillna(False)
    out = df[out]
    if len(out) == 0:
        log.warning("Safety gates removed all rows; keeping original frame to avoid empty selection.")
        return df
    return out

# -------------------------------- significance / FDR --------------------------------

def should_switch(diff_series: pd.Series) -> bool:
    """Fallback rule when no p-value: require >5% mean edge."""
    diff = pd.to_numeric(diff_series, errors="coerce").dropna()
    if len(diff) == 0:
        return True
    return diff.mean() > 0.05

def diff_series_for_ticker(ticker: str, inc_model: str, chal_model: str, df_win: pd.DataFrame) -> pd.Series:
    sub = df_win[df_win["Ticker"] == ticker] if "Ticker" in df_win.columns else df_win[df_win["Symbol"] == ticker]
    if sub.empty:
        return pd.Series(dtype=float)
    base_col = ("Score" if "Score" in sub.columns else
                ("Sharpe" if "Sharpe" in sub.columns else
                 ("Return" if "Return" in sub.columns else None)))
    if base_col is None:
        return pd.Series(dtype=float)
    idx = pd.to_datetime(sub["EndDate"], errors="coerce") if "EndDate" in sub.columns else sub.index
    piv = (sub[sub["Model"].isin([inc_model, chal_model])]
           .assign(_idx=idx)
           .pivot_table(index="_idx", columns="Model", values=base_col, aggfunc="first")
           .sort_index())
    if inc_model not in piv.columns or chal_model not in piv.columns:
        return pd.Series(dtype=float)
    return (piv[chal_model] - piv[inc_model]).dropna()

def softmax(x: np.ndarray) -> np.ndarray:
    x = np.array(x, dtype=float)
    if not np.isfinite(x).any():
        return np.ones_like(x) / len(x)
    z = x - np.nanmax(x)
    e = np.exp(z)
    s = np.nansum(e)
    return e / s if s > 0 else np.ones_like(x) / len(x)

def fdr_bh(pvals: pd.Series, alpha=0.10) -> float:
    """Return BH cutoff; p <= cutoff are accepted. If none, returns -1."""
    p = pvals.dropna().sort_values()
    m = len(p)
    if m == 0:
        return -1
    thresh = (np.arange(1, m+1) / m) * alpha
    passed = p.values <= thresh
    cutoff = p.values[passed].max() if passed.any() else -1
    return cutoff

# -------------------------------- selection / export --------------------------------

def build_selection(df_sorted: pd.DataFrame,
                    df_win: Optional[pd.DataFrame],
                    incumbents: Dict[str, Any],
                    eps: float,
                    eps_mode: str = "relative",
                    ensemble_multi: bool = False,
                    fdr_alpha: float = 0.0,
                    wilcoxon_alpha: float = 0.10,
                    ensemble_max: int = 5) -> Tuple[pd.DataFrame, Dict[str, Any]]:

    """
    Build per-ticker selection with ensemble ties and optional incumbent significance + FDR control.
    Returns (chosen_rows_df, selection_json_map).
    """
    # Pass 1: compute proposed switches and p-values
    pvals = {}
    rows_by_ticker = {}

    for ticker, tdf in df_sorted.groupby("Ticker"):
        tdf = tdf.sort_values("Score", ascending=False).reset_index(drop=True)
        rows_by_ticker[str(ticker)] = tdf
        top = tdf.iloc[0]
        inc = incumbents.get(str(ticker), {}) if isinstance(incumbents, dict) else {}
        inc_model = inc.get("model") or inc.get("Model")

        if df_win is not None and inc_model and str(inc_model) != str(top["Model"]):
            ds = diff_series_for_ticker(str(ticker), str(inc_model), str(top["Model"]), df_win)
            p = None
            if len(ds) >= 5:
                try:
                    from scipy.stats import wilcoxon
                    _, p = wilcoxon(ds)
                except Exception:
                    p = None
            if p is not None and ds.mean() > 0:
                pvals[str(ticker)] = float(p)
            else:
                pvals[str(ticker)] = np.nan

    cutoff = None
    if fdr_alpha and len([v for v in pvals.values() if pd.notna(v)]) > 0:
        cutoff = fdr_bh(pd.Series(pvals), alpha=float(fdr_alpha))
        log.info("FDR BH cutoff (alpha=%.3f): %s", fdr_alpha, cutoff)

    # Pass 2: finalize selections
    chosen_rows = []
    out_map: Dict[str, Any] = {}

    for ticker, tdf in rows_by_ticker.items():
        tdf = tdf.sort_values("Score", ascending=False).reset_index(drop=True)
        top = tdf.iloc[0]
        second = tdf.iloc[1] if len(tdf) > 1 else None

        top_score = float(top["Score"])
        thr = float(eps) if eps_mode == "absolute" else max(0.0, abs(top_score) * float(eps))

        # Ensemble logic
        selection: Dict[str, Any] = {"mode": "single"}
        if ensemble_multi:
            within = tdf[(top_score - tdf["Score"]) <= thr].copy()
            within = within[np.isfinite(within["Score"])]
            if len(within) >= 2:
                within = within.sort_values("Score", ascending=False).head(ensemble_max)
                models = within["Model"].astype(str).tolist()
                w = softmax(within["Score"].values)
                selection = {"mode": "ensemble", "members": models, "weights": [float(x) for x in w.tolist()]}
        else:
            if second is not None and (top_score - float(second["Score"])) <= thr:
                selection = {"mode": "ensemble", "secondary": str(second["Model"])}

        chosen_row = top
        chosen_model = str(top["Model"])

        # Significance guard vs incumbent (with optional FDR)
        inc = incumbents.get(ticker, {})
        inc_model = inc.get("model") or inc.get("Model")
        if inc_model and inc_model != chosen_model and df_win is not None:
            ds = diff_series_for_ticker(ticker, str(inc_model), chosen_model, df_win)
            allow_switch = False
            used_p = None
            if len(ds) >= 5:
                try:
                    from scipy.stats import wilcoxon
                    _, p = wilcoxon(ds)
                    used_p = float(p)
                    if ds.mean() > 0 and p <= wilcoxon_alpha:
                        allow_switch = True
                        if cutoff is not None and not np.isnan(cutoff):
                            allow_switch = allow_switch and (p <= cutoff)
                except Exception:
                    allow_switch = should_switch(ds)
            else:
                allow_switch = should_switch(ds)

            if not allow_switch:
                if (tdf["Model"] == inc_model).any():
                    chosen_row = tdf[tdf["Model"] == inc_model].iloc[0]
                    chosen_model = str(inc_model)
                    note = "kept_incumbent"
                    if used_p is not None:
                        note += f"_p={used_p:.3f}"
                        if cutoff is not None and not np.isnan(cutoff):
                            note += f"_bh_cutoff={cutoff:.3f}"
                    selection = {"mode": "single", "note": note}

        # Stability & artifacts (if available)
        maxdd = (chosen_row["Max_Drawdown"] if "Max_Drawdown" in chosen_row and pd.notna(chosen_row["Max_Drawdown"])
                 else (chosen_row["Drawdown"] if "Drawdown" in chosen_row else None))

        stability = {}
        if "Recent_Wins_3mo" in tdf.columns:
            val = pd.to_numeric(chosen_row.get("Recent_Wins_3mo", np.nan), errors="coerce")
            stability["win_rate_k_of_n"] = float(val) if pd.notna(val) else 0.0
        if "Sharpe_Var" in tdf.columns:
            val = pd.to_numeric(chosen_row.get("Sharpe_Var", np.nan), errors="coerce")
            stability["sharpe_var"] = float(val) if pd.notna(val) else None

        artifact = {}
        exists = {}
        for k, col in [("path","artifact_path"), ("vecnorm","vecnorm_path"), ("features","features_path")]:
            if col in tdf.columns:
                artifact[k] = chosen_row.get(col)
                if isinstance(artifact[k], str):
                    exists[k] = Path(artifact[k]).exists()
        if artifact:
            if "load_ms" in tdf.columns:
                artifact["load_ms"] = int(pd.to_numeric(chosen_row.get("load_ms", 0), errors="coerce") or 0)
            if "mem_mb" in tdf.columns:
                artifact["mem_mb"] = int(pd.to_numeric(chosen_row.get("mem_mb", 0), errors="coerce") or 0)
            if exists:
                artifact["exists"] = exists

        out_map[str(ticker)] = {
            "model": chosen_model,
            "family": infer_family_from_model(str(chosen_model)),
            "regime": (str(chosen_row.get("regime")) if "regime" in chosen_row else None),
            "rl_profile": (str(chosen_row.get("rl_profile")) if "rl_profile" in chosen_row else None),
            "score": float(chosen_row.get("Score", 0.0)),
            "sharpe": float(pd.to_numeric(chosen_row.get("Sharpe", np.nan), errors="coerce")) if "Sharpe" in chosen_row else None,
            "return": float(pd.to_numeric(chosen_row.get("Return", np.nan), errors="coerce")) if "Return" in chosen_row else None,
            "max_drawdown": float(pd.to_numeric(maxdd, errors="coerce")) if maxdd is not None else None,
            "sortino": float(pd.to_numeric(chosen_row.get("Sortino_Mean", np.nan), errors="coerce")) if "Sortino_Mean" in chosen_row else None,
            "trade_count": int(pd.to_numeric(chosen_row.get("Trade_Count", np.nan), errors="coerce")) if "Trade_Count" in chosen_row else None,
            "turnover": float(pd.to_numeric(chosen_row.get("Turnover", np.nan), errors="coerce")) if "Turnover" in chosen_row else None,
            "stability": stability if stability else None,
            "artifact": artifact if artifact else None,
            "selection": selection,
        }
        chosen_rows.append(chosen_row)

    chosen_df = pd.DataFrame(chosen_rows).reset_index(drop=True)
    return chosen_df, out_map

def summarize_by_model(df: pd.DataFrame) -> pd.DataFrame:
    summary_cols = [c for c in ["Score", "Sharpe", "Return", "Accuracy", "Drawdown", "Final_Portfolio"] if c in df.columns]
    agg = df.groupby("Model", as_index=False)[summary_cols].mean(numeric_only=True)
    return agg.sort_values("Score", ascending=False) if "Score" in agg.columns else agg

def pick_best_per_ticker(df: pd.DataFrame, k: int = 1) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if "Ticker" not in df.columns and "Symbol" in df.columns:
        df = df.rename(columns={"Symbol": "Ticker"})
    if "Ticker" in df.columns:
        df["Ticker"] = df["Ticker"].astype(str).str.upper().str.strip()
        df["Model"] = df["Model"].astype(str).str.strip()
    if "Ticker" not in df.columns:
        raise ValueError("Expected 'Ticker' or 'Symbol' in input CSV.")

    sort_cols, sort_asc = [], []
    def add(col, asc=False):
        if col in df.columns:
            sort_cols.append(col); sort_asc.append(asc)

    add("Score", asc=False)
    add("Sharpe", asc=False)
    add("Return", asc=False)
    add("Sortino_Mean", asc=False)
    add("Drawdown", asc=True)
    add("MaxDD_Mean", asc=True)
    add("N_Windows", asc=False)
    add("Turnover", asc=True)
    add("Accuracy", asc=False)
    add("Final_Portfolio", asc=False)

    if not sort_cols:
        raise ValueError("No comparable columns to sort by. Provide metrics or weights.")

    df_sorted = df.sort_values(["Ticker"] + sort_cols, ascending=[True] + sort_asc)

    if k == 1:
        best = df_sorted.groupby("Ticker", as_index=False).first()
    else:
        best = df_sorted.groupby("Ticker").head(k).reset_index(drop=True)
    return best, df_sorted

def write_outputs(best: pd.DataFrame, outdir: Path, tag: str, sel_map: Dict[str, Any]):
    outdir.mkdir(parents=True, exist_ok=True)
    csv_path = outdir / f"model_selector_summary_{tag}.csv"
    xlsx_path = outdir / f"model_selector_summary_{tag}.xlsx"
    json_path = outdir / f"selected_models_{tag}.json"

    best.to_csv(csv_path, index=False)
    try:
        best.to_excel(xlsx_path, index=False)
    except Exception as e:
        log.warning("Excel write failed (%s); continuing with CSV+JSON.", e)

    with open(json_path, "w") as f:
        json.dump(sel_map, f, indent=2)

    return str(csv_path), str(xlsx_path), str(json_path)

def write_meta(outdir: Path, args):
    meta = {
        "timestamp": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "version": "4.1.x",
        "args": vars(args)
    }
    try:
        (outdir / "run_meta.json").write_text(json.dumps(meta, indent=2))
    except Exception as e:
        log.warning("Failed to write run_meta.json (%s)", e)

def apply_family_cap(df_sorted: pd.DataFrame, cap: int) -> pd.DataFrame:
    if cap <= 0 or "Family" not in df_sorted.columns:
        return df_sorted
    keep = []
    for _, g in df_sorted.groupby("Ticker", sort=False):
        counts: Dict[str,int] = {}
        for _, r in g.iterrows():
            f = r["Family"]
            counts[f] = counts.get(f, 0) + 1
            if counts[f] <= cap:
                keep.append(r)
    return pd.DataFrame(keep)

# -------------------------------- pipeline --------------------------------

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--input", required=True, help="Path to combined CSV (all_models_selector_ready.csv)")
    ap.add_argument("--outdir", required=True, help="Output directory for summary files")

    # Scoring & scaling
    ap.add_argument("--weights", default="Sharpe=0.5,Return=0.3,Accuracy=0.2",
                    help="Comma key=val list, e.g. 'Sharpe=0.5,Return=0.3,Accuracy=0.2,Drawdown=-0.1'")
    ap.add_argument("--scale", choices=["z", "robust", "none"], default="z", help="Metric scaling before weighting")
    ap.add_argument("--winsor", type=float, default=0.0, help="Winsorization p in [0,0.1]; try 0.01")

    # Data handling
    ap.add_argument("--low_memory", action="store_true", help="Pass low_memory to pandas read_csv")
    ap.add_argument("--dtypes_path", default="", help="Optional JSON file with column->dtype for read_csv")
    ap.add_argument("--drop_nan", action="store_true", help="Drop rows with NaNs in key metrics")

    # History / recency / stability
    ap.add_argument("--windows", default=None, help="(Optional) per-window history CSV for stability/recency/significance")
    ap.add_argument("--lambda_decay", type=float, default=0.02, help="Recency exponential decay per day")
    ap.add_argument("--halflife_days", type=float, default=0.0, help="If >0, overrides lambda via ln(2)/half_life")
    ap.add_argument("--sharpe_se_alpha", type=float, default=0.5, help="Penalty factor for Sharpe standard error")

    # Costs & gates
    ap.add_argument("--cost_bps", type=float, default=0.5, help="Turnover cost in basis points")
    ap.add_argument("--min_windows", type=int, default=3, help="Minimum number of windows required")
    ap.add_argument("--min_trades", type=float, default=1.0, help="Minimum average trades required")
    ap.add_argument("--gates", default="", help='Custom gates, e.g. "Sharpe>0,Max_Drawdown>-0.5,Return>-0.2"')
    ap.add_argument("--flip_dd_positive", action="store_true",
                    help="If drawdowns are mostly positive values, flip sign to negative.")

    # Ensemble / Incumbents / FDR
    ap.add_argument("--eps", type=float, default=0.03, help="Tie epsilon")
    ap.add_argument("--eps_mode", choices=["relative","absolute"], default="relative",
                    help="Use relative (fraction of top score) or absolute EPS for ensembling")
    ap.add_argument("--ensemble_multi", action="store_true",
                    help="If set, include all models within EPS threshold (up to 5)")
    ap.add_argument("--ensemble_max", type=int, default=5, help="Max models in multi-ensemble")
    ap.add_argument("--incumbents", default="", help="(Optional) Path to prior selected_models_*.json for significance guard")
    ap.add_argument("--fdr_alpha", type=float, default=0.0, help="Benjamini–Hochberg FDR alpha (0=disable)")

    # Universal / family
    ap.add_argument("--topk", type=int, default=1, help="Top-K rows per ticker for CSV/XLSX table")
    ap.add_argument("--annualize", action="store_true", help="Annualize Sharpe/Return using PeriodDays if present")
    ap.add_argument("--period_col", default="PeriodDays", help="Column with period length (days)")
    ap.add_argument("--trading_days", type=int, default=252, help="Trading days per year for annualization")
    ap.add_argument("--max_latency_ms", type=int, default=0, help="Drop models with latency_ms above this (0=ignore)")
    ap.add_argument("--max_mem_mb", type=int, default=0, help="Drop models with mem_mb above this (0=ignore)")
    ap.add_argument("--require_artifacts", action="store_true", help="Keep only rows with non-null artifact paths")
    ap.add_argument("--family_weights", default="",
                    help='JSON map: family -> weight spec. e.g. '
                         '\'{"RL":"Sharpe=0.6,Return=0.3,Drawdown=-0.1","Tree":"Sharpe=0.5,Return=0.4,Turnover=-0.1"}\'')
    ap.add_argument("--family_cap", type=int, default=0, help="Cap models per family per ticker before selection (0=off)")

    # Exports / ops
    ap.add_argument("--export_sorted", default="", help="Write full sorted table CSV here")
    ap.add_argument("--export_parquet", default="", help="Write full sorted table Parquet here")
    ap.add_argument("--debug_csv", default="", help="Write intermediate scored DF for audit (path)")
    ap.add_argument("--dry_run", action="store_true", help="Compute and validate—do not write outputs")

    # --- your requested try/except around parse_args() ---
    try:
        args = ap.parse_args()
    except SystemExit as e:
        raise RuntimeError("Missing required args: --input and --outdir") from e

    # Half-life → lambda (if provided)
    if args.halflife_days and args.halflife_days > 0:
        args.lambda_decay = np.log(2.0) / float(args.halflife_days)
        log.info("Using lambda_decay from half-life: %.6f (half-life=%.1f days)", args.lambda_decay, args.halflife_days)

    weights = parse_weights(args.weights)
    fam_w = parse_family_weights(args.family_weights)
    tag = datetime.now().strftime("%Y-%m-%d")
    log.info("Scaling mode: %s", args.scale)

    # Read CSV with optional dtype hints
    read_kwargs: Dict[str, Any] = {"low_memory": args.low_memory}
    if args.dtypes_path and Path(args.dtypes_path).exists():
        try:
            dtypes = json.loads(Path(args.dtypes_path).read_text())
            read_kwargs["dtype"] = dtypes
        except Exception as e:
            log.warning("Failed to load dtypes JSON (%s); continuing without.", e)

    log.info("Reading input: %s", args.input)
    df = pd.read_csv(args.input, **read_kwargs)
    validate_schema(df)
    log.info("Shape: %s, columns (first 12): %s", df.shape, ", ".join(df.columns[:12]))

    # Normalize ticker
    if "Ticker" not in df.columns and "Symbol" in df.columns:
        df = df.rename(columns={"Symbol": "Ticker"})
    df["Ticker"] = df["Ticker"].astype(str).str.upper().str.strip()

    # Family + harmonization + annualization + resource gates
    df["Family"] = df["Model"].astype(str).apply(infer_family_from_model)
    df = harmonize_accuracy_like(df)
    if args.annualize:
        df = annualize_metrics(df, args.period_col, args.trading_days)
    df = apply_resource_gates(df, args.require_artifacts, args.max_latency_ms, args.max_mem_mb)

    # Optional normalize drawdown sign
    # (use --flip_dd_positive if drawdowns are mostly positive and should be negative)
    if args.flip_dd_positive:
        df = normalize_drawdown_sign(df)

    # ===== v4.1 history-based stability + recency scoring (optional) =====
    df_win = None
    if args.windows:
        log.info("Reading windows history: %s", args.windows)
        log.info("Windows composites using %s z-scores", "robust" if args.scale == "robust" else "standard")
        df_win = pd.read_csv(args.windows, **({"low_memory": args.low_memory} if args.low_memory else {}))

        if "Ticker" not in df_win.columns and "Symbol" in df_win.columns:
            df_win = df_win.rename(columns={"Symbol": "Ticker"})
        if "Max_Drawdown" not in df_win.columns and "Drawdown" in df_win.columns:
            df_win["Max_Drawdown"] = df_win["Drawdown"]

        for col in ["Sharpe", "Return", "Max_Drawdown", "Sortino", "Trade_Count", "Turnover", "Is_Win_Recent"]:
            if col in df_win.columns:
                df_win[col] = pd.to_numeric(df_win[col], errors="coerce")

        # Aggregates across windows
        agg = (df_win
               .groupby(["Ticker", "Model"])
               .agg(Sharpe_Mean=("Sharpe", "mean"),
                    Sharpe_Var=("Sharpe", "var"),
                    Return_Mean=("Return", "mean"),
                    MaxDD_Mean=("Max_Drawdown", "mean"),
                    Sortino_Mean=("Sortino", "mean"),
                    Trade_Count=("Trade_Count", "mean"),
                    Turnover=("Turnover", "mean"),
                    Recent_Wins_3mo=("Is_Win_Recent", "mean"))
               .reset_index())

        # Count windows + sample-size penalty on Sharpe
        n_win = (df_win.groupby(["Ticker", "Model"])["Sharpe"]
                      .size()
                      .reset_index(name="N_Windows"))
        agg = agg.merge(n_win, on=["Ticker", "Model"], how="left")
        agg["Sharpe_SE"] = np.sqrt((1.0 + 0.5 * np.square(agg["Sharpe_Mean"])) / agg["N_Windows"].clip(lower=1))
        agg["Sharpe_Mean_adj"] = agg["Sharpe_Mean"] - float(args.sharpe_se_alpha) * agg["Sharpe_SE"]

        # Recency weights
        now = pd.Timestamp.utcnow()
        if "EndDate" in df_win.columns:
            df_win["age_days"] = (now - pd.to_datetime(df_win["EndDate"], utc=True)).dt.days.clip(lower=0)
        else:
            df_win["age_days"] = 0
        df_win["rec_w"] = np.exp(-float(args.lambda_decay) * df_win["age_days"].astype(float))

        rec = (df_win.groupby(["Ticker", "Model"])
                     .apply(lambda g: pd.Series({
                         "Sharpe_Rec": (g["Sharpe"] * g["rec_w"]).sum() / max(1e-9, g["rec_w"].sum()),
                         "Return_Rec": (g["Return"] * g["rec_w"]).sum() / max(1e-9, g["rec_w"].sum()),
                     }))
                     .reset_index())

        # Merge into latest-window frame
        df_ext = (df.merge(agg, on=["Ticker", "Model"], how="left")
                    .merge(rec, on=["Ticker", "Model"], how="left"))

        # Fallbacks / costs / stability
        if "Sortino_Mean" not in df_ext.columns or df_ext["Sortino_Mean"].isna().all():
            df_ext["Sortino_Mean"] = df_ext.get("Sharpe", 0)

        df_ext["Turnover"] = pd.to_numeric(df_ext.get("Turnover", 0), errors="coerce").fillna(0)
        df_ext["Sharpe_Var"] = pd.to_numeric(df_ext.get("Sharpe_Var", 0), errors="coerce").fillna(0)
        df_ext["Recent_Wins_3mo"] = pd.to_numeric(df_ext.get("Recent_Wins_3mo", 0), errors="coerce").fillna(0)
        df_ext["CostPenalty"] = df_ext["Turnover"] * (float(args.cost_bps) / 10000.0)
        df_ext["StabilityBonus"] = (0.10 * df_ext["Recent_Wins_3mo"]) - (0.05 * df_ext["Sharpe_Var"])

        # z-scores (single pass; robust optional)
        def _z_meanstd(s):
            s = pd.to_numeric(s, errors="coerce").fillna(0)
            mu, sd = s.mean(), s.std(ddof=0)
            return (s - mu) / sd if np.isfinite(sd) and sd != 0 else pd.Series(0.0, index=s.index)

        z_fun = robust_z if (hasattr(args, "scale") and args.scale == "robust") else _z_meanstd
        for c in ["Sharpe_Mean_adj", "Sharpe_Rec", "Return_Rec", "Sortino_Mean", "MaxDD_Mean"]:
            if c in df_ext.columns:
                df_ext[f"z_{c}"] = z_fun(df_ext[c])

        # Composite score (prefers adjusted Sharpe if present)
        df_ext["Score"] = (
            0.45 * df_ext.get("z_Sharpe_Mean_adj", df_ext.get("z_Sharpe_Rec", 0)) +
            0.25 * df_ext.get("z_Return_Rec", 0) +
            0.15 * df_ext.get("z_Sortino_Mean", 0) -
            0.15 * df_ext.get("z_MaxDD_Mean", 0) -
            df_ext.get("CostPenalty", 0) +
            df_ext.get("StabilityBonus", 0)
        )

        # Minimum-data gates
        df_ext = df_ext[df_ext["N_Windows"].fillna(0) >= args.min_windows]
        df_ext = df_ext[df_ext["Trade_Count"].fillna(0) >= args.min_trades]

        df = df_ext

    else:
        # Original path (no windows file): family-aware scoring
        df = compute_score_frame_with_family_weights(
            df,
            default_weights=weights,
            family_weights=fam_w,
            scale_mode=args.scale,
            winsor_p=args.winsor
        )
        # Turnover cost (if present)
        if "Turnover" in df.columns:
            df["Score"] = pd.to_numeric(df["Score"], errors="coerce").fillna(0) - \
                          pd.to_numeric(df["Turnover"], errors="coerce").fillna(0) * (float(args.cost_bps)/10000.0)

    # Optional NaN drop on key fields used for sorting
    keys = [c for c in ["Score", "Sharpe", "Return", "Accuracy", "Drawdown", "Final_Portfolio"] if c in df.columns]
    if args.drop_nan and keys:
        df = df.dropna(subset=keys)

    # Safety + custom gates + hygiene
    before_rows = len(df)
    df = apply_safety_gates(df)
    df = apply_gates_expr(df, args.gates)
    df = dedupe(df)
    df["Score"] = pd.to_numeric(df["Score"], errors="coerce").replace([np.inf,-np.inf], np.nan)
    if df["Score"].isna().any():
        fill_val = (df["Score"].min(skipna=True) or 0.0) - 1e6
        df["Score"] = df["Score"].fillna(fill_val)
    log.info("Rows after gates: %d (from %d)", len(df), before_rows)

    # Sort for selection/tiebreakers
    sort_cols, sort_asc = [], []
    def add(col, asc=False):
        if col in df.columns:
            sort_cols.append(col); sort_asc.append(asc)
    add("Score", False); add("Sharpe", False); add("Return", False)
    add("Sortino_Mean", False); add("Drawdown", True); add("Max_Drawdown", True)
    add("Accuracy", False); add("Final_Portfolio", False); add("Turnover", True)

    if not sort_cols:
        raise SystemExit("No comparable columns to sort by. Provide metrics or weights.")

    df_sorted = df.sort_values(["Ticker"] + sort_cols, ascending=[True] + sort_asc)
    df_sorted = apply_family_cap(df_sorted, args.family_cap)

    # Ensure outdir exists before quick-peek write
    Path(args.outdir).mkdir(parents=True, exist_ok=True)

    # Quick peek of the top 100 rows (optional)
    try:
        (Path(args.outdir) / f"quick_peek_top100_{tag}.csv").write_text(
            df_sorted.head(100).to_csv(index=False)
        )
        log.info("Wrote quick_peek_top100_%s.csv", tag)
    except Exception as e:
        log.debug("quick_peek export skipped: %s", e)

    if args.export_sorted:
        try:
            Path(args.export_sorted).parent.mkdir(parents=True, exist_ok=True)
            df_sorted.to_csv(args.export_sorted, index=False)
            log.info("Wrote full sorted table: %s", args.export_sorted)
        except Exception as e:
            log.warning("Failed to export sorted table (%s)", e)

    if args.export_parquet:
        try:
            Path(args.export_parquet).parent.mkdir(parents=True, exist_ok=True)
            df_sorted.to_parquet(args.export_parquet, index=False)
            log.info("Wrote parquet: %s", args.export_parquet)
        except Exception as e:
            log.warning("Parquet export failed (install pyarrow or fastparquet?): %s", e)

    # Load incumbents JSON (optional)
    incumbents: Dict[str, Any] = {}
    if args.incumbents and Path(args.incumbents).exists():
        try:
            with open(args.incumbents, "r") as f:
                incumbents = json.load(f)
                incumbents = {str(k).upper().strip(): v for k, v in incumbents.items()}
        except Exception as e:
            log.warning("Failed to load incumbents JSON: %s", e)

    # JSON (ensemble/significance + optional FDR) using df_sorted
    _, selection_map = build_selection(
        df_sorted=df_sorted,
        df_win=df_win,
        incumbents=incumbents,
        eps=float(args.eps),
        eps_mode=args.eps_mode,
        ensemble_multi=args.ensemble_multi,
        fdr_alpha=float(args.fdr_alpha),
        ensemble_max=int(args.ensemble_max),
    )

    # Top-K table for CSV/XLSX
    best_pk, _ = pick_best_per_ticker(df, k=args.topk)

    # Summary by model (info log)
    try:
        model_means = summarize_by_model(df)
        log.info("Top models (means):\n%s", model_means.to_string(index=False))
    except Exception:
        pass

    if args.dry_run:
        log.info("Dry run: skipping writes")
        return

    # Write outputs (CSV/XLSX + richer JSON) + provenance
    outdir = Path(args.outdir)
    csv_path, xlsx_path, json_path = write_outputs(best_pk, outdir, tag, selection_map)
    write_meta(outdir, args)

    log.info("Saved: %s", csv_path)
    log.info("Saved: %s", xlsx_path)
    log.info("Saved: %s", json_path)

# =========================
# STEP C: RUN PIPELINE
# =========================

# In Colab, we simulate CLI flags by setting sys.argv before calling main().
# If you want to tweak parameters, edit the list below.
if IN_COLAB:
    os.makedirs(OUTDIR, exist_ok=True)
    sys.argv = [
        "colab_kernel_launcher.py",
        "--input", COMBINED_CSV,
        "--outdir", OUTDIR,
        # Optional tuning flags (uncomment or edit as needed):
        # "--weights", "Sharpe=0.5,Return=0.3,Accuracy=0.2,Drawdown=-0.1",
        # "--scale", "robust",
        # "--winsor", "0.01",
        # "--gates", "Sharpe>0,Max_Drawdown>-0.5,Return>-0.2",
        # "--family_cap", "3",
        # "--ensemble_multi",
        # "--eps_mode", "relative",
        # "--eps", "0.03",
        # "--export_sorted", os.path.join(OUTDIR, "full_sorted.csv"),
        "--dry_run"  # set to comment/remove to actually write the final outputs
    ]

# Normal script guard (lets this file also run from the command-line)
if __name__ == "__main__":
    main()

# =========================
# STEP D: OPTIONAL — PEEK AT BEST MODELS (from dry_run data frame path)
# If you removed --dry_run above, the outputs will be written under OUTDIR.
# =========================

print("\n✅ Pipeline completed. Check your output folder here:")
print(OUTDIR)


Mounted at /content/drive


ValueError: No model selector CSVs found in folder. Place files ending with *_model_selector.csv into SELECTOR_DIR.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# === Ensure numeric types for plotting ===
for col in ['Sharpe', 'Return', 'Accuracy', 'Score']:
    df[col] = pd.to_numeric(df.get(col), errors='coerce').fillna(0)

# === Compute model averages ===
model_means = df.groupby("Model")[['Sharpe', 'Return', 'Accuracy', 'Score']].mean().sort_values("Score", ascending=False)

# === Select Top 5 and Bottom 5 Models ===
top5_models = model_means.head(5)
bottom5_models = model_means.tail(5)

# === Combine for Top & Bottom Chart ===
top_bottom_combined = pd.concat([top5_models, bottom5_models])

# === Plot 1: Top & Bottom 5 Models by Average Score ===
plt.figure(figsize=(14, 6))
top_bottom_combined[['Score']].sort_values("Score").plot(kind='barh', legend=False, color='steelblue')
plt.title("Top 5 and Bottom 5 Models by Average Score")
plt.xlabel("Average Score")
plt.grid(True)
plt.tight_layout()
plt.show()

# === Plot 2: Average Metric Comparison (Top 5 only) ===
plt.figure(figsize=(12, 6))
top5_models[['Sharpe', 'Return', 'Accuracy']].plot(kind='bar', figsize=(14, 6))
plt.title("Average Sharpe, Return, and Accuracy for Top 5 Models")
plt.ylabel("Average Value")
plt.xticks(rotation=0)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# === Plot 3: Score Distribution per Model ===
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="Model", y="Score", hue="Model", palette="Set3", dodge=False)
plt.title("Score Distribution per Model")
plt.legend([],[], frameon=False)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === Plot 4: Return Distribution per Model ===
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="Model", y="Return", hue="Model", palette="Set2", dodge=False)
plt.title("Return Distribution per Model")
plt.legend([],[], frameon=False)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# === Plot 5: Sharpe Ratio Distribution per Model ===
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="Model", y="Sharpe", hue="Model", palette="Set1", dodge=False)
plt.title("Sharpe Ratio Distribution per Model")
plt.legend([],[], frameon=False)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
